<a href="https://colab.research.google.com/github/Igorvl/Projects/blob/master/%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D1%85_%D1%81%D0%B5%D1%82%D0%B5%D0%B9_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Создать 9 нейросетей с различными параметрами словаря и гиперпараметрами нейронной сети. (см. пункт 2 и 3) 

 Для этого:

  1. Загрузить необходимую базу. https://storage.yandexcloud.net/aiueducation/Content/base/l7/writers.zip

  2. Параметры размера словаря, ширины окна и шага:

    - Размер словаря - от 10000 до 20000 (выбрать меньшее значение диапазона, если будет перегрузка ОЗУ и перезапуск подключения к Colaboratory)
    - Ширина окна - от 1000 до 2000
    - Шаг - от 100 до 500 (меньший шаг лучше для  обучения, но это может перегрузить ОЗУ).

  3. Архитектура сети с гиперпараметрами. Можно воспользоваться шаблоном:
  
   - Модель прямого распространения **Sequential()**
   - Один или несколько полносвязных (**Dense**) слоёв
   - Слои **Dropout()** и **BatchNormalization()**
   - Выходной полносвязный слой с количеством нейронов, соответствующим количеству классов (число писателей)
  
   Точность сети можно проверить по значению показателя 'val_accuracy' на конце каждой эпохи. 
   

Загрузка библиотек и базы данных.

Подключаем необходимые библиотеки и модули:

In [1]:
# Работа с массивами данных
import numpy as np 

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation 

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Матрица ошибок классификатора
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Работа с google диском
from google.colab import drive

# Загрузка датасетов из облака google
import gdown

# Функции операционной системы
import os

# Регулярные выражения
import re

Загрузка датасета с текстами писателей в виде архива и распаковка его в папку:

In [2]:
# Загрузка датасета из облака
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l7/writers.zip', None, quiet=True)

# Распаковка архива в папку writers
!unzip -qo writers.zip -d writers/

# Просмотр содержимого папки
!ls writers

'(Булгаков) Обучающая_5 вместе.txt'
'(Булгаков) Тестовая_2 вместе.txt'
'(Клиффорд_Саймак) Обучающая_5 вместе.txt'
'(Клиффорд_Саймак) Тестовая_2 вместе.txt'
'(Макс Фрай) Обучающая_5 вместе.txt'
'(Макс Фрай) Тестовая_2 вместе.txt'
'(О. Генри) Обучающая_50 вместе.txt'
'(О. Генри) Тестовая_20 вместе.txt'
'(Рэй Брэдберри) Обучающая_22 вместе.txt'
'(Рэй Брэдберри) Тестовая_8 вместе.txt'
'(Стругацкие) Обучающая_5 вместе.txt'
'(Стругацкие) Тестовая_2 вместе.txt'


Сохранение в отдельных переменных:

   - Имя для папки с текстами
   - Название выборки "обучающая"
   - Название выборки "тестовая"

In [3]:
# Папка с текстовыми файлами
FILE_DIR  = 'writers'

# Признак обучающей выборки в имени файла                     
SIG_TRAIN = 'обучающая'

# Признак тестовой выборки в имени файла                   
SIG_TEST  = 'тестовая'                    

**Шаг 2.** Преобразование базы данных для обучения.

Добавление имен всех писателей в список классов.

Преобразование текстов в одну длинную строку и разбиение их отдельно в списки для каждого класса и выборки.

In [4]:
# В список добавляются имена классов
CLASS_LIST = []

# Здесь сохраняются тексты для обучения сети
text_train = []

# А здесь для проверки точности сети
text_test = []
 
# цикл для итерации по каждому имени текста в общей папке
for file_name in os.listdir(FILE_DIR):

    # Выделение имени класса и типа выборки из имени файла
    m = re.match('\((.+)\) (\S+)_', file_name)

    # Если выделение получилось, то файл обрабатывается
    if m:

        # отдельно берём имя класса (автора)
        class_name = m[1]

        # отдельно - имя выборки
        subset_name = m[2].lower()

        # Проверка типа выборки по названию в имени файла
        is_train = SIG_TRAIN in subset_name
        is_test = SIG_TEST in subset_name

        # Если тип выборки обучающая или тестовая - файл обрабатывается
        if is_train or is_test:

            # Добавление нового класса, если его еще нет в списке
            if class_name not in CLASS_LIST:

                # Выводится информационное сообщение о добавлении названия класса
                CLASS_LIST.append(class_name)

                # Инициализация соответствующих классу строк текста
                text_train.append('')
                text_test.append('')

            # Поиск индекса класса для добавления содержимого файла в выборку
            cls = CLASS_LIST.index(class_name)

            # Выводится информационное сообщение о добавлении класса в список классов и текста к выборке
            print(f'Добавление файла "{file_name}" в класс "{CLASS_LIST[cls]}", {subset_name} выборка.')

            # оператор with - безопасное чтение каждого файла с текстом
            with open(f'{FILE_DIR}/{file_name}', 'r') as f:

                # Загрузка содержимого файла в строку
                text = f.read()

            # Определение выборки, куда будет добавлено содержимое
            subset = text_train if is_train else text_test

            # Добавление текста к соответствующей выборке класса. Концы строк заменяются на пробел
            subset[cls] += ' ' + text.replace('\n', ' ')
            
# Определим кол-во классов
CLASS_COUNT = len(CLASS_LIST)

Добавление файла "(Стругацкие) Тестовая_2 вместе.txt" в класс "Стругацкие", тестовая выборка.
Добавление файла "(Клиффорд_Саймак) Обучающая_5 вместе.txt" в класс "Клиффорд_Саймак", обучающая выборка.
Добавление файла "(Булгаков) Тестовая_2 вместе.txt" в класс "Булгаков", тестовая выборка.
Добавление файла "(О. Генри) Тестовая_20 вместе.txt" в класс "О. Генри", тестовая выборка.
Добавление файла "(Рэй Брэдберри) Тестовая_8 вместе.txt" в класс "Рэй Брэдберри", тестовая выборка.
Добавление файла "(Макс Фрай) Тестовая_2 вместе.txt" в класс "Макс Фрай", тестовая выборка.
Добавление файла "(Стругацкие) Обучающая_5 вместе.txt" в класс "Стругацкие", обучающая выборка.
Добавление файла "(Булгаков) Обучающая_5 вместе.txt" в класс "Булгаков", обучающая выборка.
Добавление файла "(О. Генри) Обучающая_50 вместе.txt" в класс "О. Генри", обучающая выборка.
Добавление файла "(Макс Фрай) Обучающая_5 вместе.txt" в класс "Макс Фрай", обучающая выборка.
Добавление файла "(Клиффорд_Саймак) Тестовая_2 вмест

Подготовка текстовой инфрмации для обработки Токенайзером. Для этого сохрание в подходящих именах переменных со следующим параметрами:

   - Объем словаря для токенизатора
   - Длина отрезка текста (окна) в словах
   - Шаг смещения окна по тексту для деления на векторы

И затем вызов самого Токенайзера, передав в его параметры эти переменные и оставшиеся значения для преобразования.

In [9]:
# Объем словаря для токенизатора
VOCAB_SIZE = 15000

# Длина отрезка текста (окна) в словах                        
WIN_SIZE   = 3000

# Шаг окна разбиения текста на векторы                         
WIN_HOP    = 300                          

# Токенайзер из Keras для разбиения текста и построения частотного словаря
tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', 
                          lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)

Отдельно обучение самого Токенайзера на выборке текстов для обучения сети:


In [10]:
# Получаем словарь частотности 
tokenizer.fit_on_texts(text_train)

Преобразование обучающих и проверочных текстов в последовательность индексов согласно частотному словарю Токенайзера:

In [11]:
seq_train = tokenizer.texts_to_sequences(text_train)
seq_test = tokenizer.texts_to_sequences(text_test)

**Шаг 3.** Создание функций для формирования выборок. Формирование выборок.

Воспроизведение блока кода с двумя функциями:

  1. Функция деления последовательности индексов на отрезки скользящим окном

In [ ]:
def split_sequence(sequence,   # Последовательность индексов
                   win_size,   # Размер окна для деления на примеры
                   hop):       # Шаг окна

    # Последовательность разбивается на части до последнего полного окна
    return [sequence[i:i + win_size] for i in range(0, len(sequence) - win_size + 1, hop)]

 2. Функция формирования выборок из индексов и соответствующих классам меток в формате One Hot Encoding

In [ ]:
def vectorize_sequence(seq_list,    # Список последовательностей индексов 
                       win_size,    # Размер окна для деления на примеры
                       hop):        # Шаг окна

    # В списке последовательности следуют в порядке их классов (их кол-во сповпадает с кол-вом классов)
    class_count = len(seq_list)

    # Списки для исходных векторов и категориальных меток класса
    x, y = [], []

    # Для каждого класса:
    for cls in range(class_count):

        # Разбиение последовательности класса cls на отрезки
        vectors = split_sequence(seq_list[cls], win_size, hop)

        # Добавление отрезков в выборку

        x += vectors
        
        # Для всех отрезков класса cls добавление меток класса в виде OHE
        y += [utils.to_categorical(cls, class_count)] * len(vectors)

    # Возврат результатов как numpy-массивов
    return np.array(x), np.array(y)

Формирование выборки (x_train, y_train) и (x_test, y_test) с помощью функций, воспроизведённых в ячейке выше:

In [ ]:
# Формирование обучающей выборки
x_train, y_train = vectorize_sequence(seq_train, WIN_SIZE, WIN_HOP) 
# Формирование тестовой выборки
x_test, y_test = vectorize_sequence(seq_test, WIN_SIZE, WIN_HOP)

# Проверка формы сформированных данных
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5842, 3000) (5842, 6)
(2190, 3000) (2190, 6)


Формирование выборки индексов слов в виде матриц Bag Of Words методом  **sequences_to_matrix( )**. 

In [ ]:
# На входе .sequences_to_matrix() ожидает список, .tolist() выполняет преобразование к типу данных 'список'
x_train_01 = tokenizer.sequences_to_matrix(x_train.tolist())
x_test_01 = tokenizer.sequences_to_matrix(x_test.tolist())

# Выводим форму обучающей выборки в виде матрицы Bag of Words
print(x_train_01.shape)       
# Выводим фрагмент отрезка обучающего текста в виде Bag of Words
print(x_train_01[0][0:100])

(5842, 15000)
[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]


**Шаг 4.** Создание архитектуры нейронной сети.

Создайте несложную архитектуру нейронной сети. После этого произведите компиляцию модели с соответствующими параметрами:

In [ ]:
model_BoW = Sequential()                                            # Создание последовательной модели нейросети
model_BoW.add(Dense(184, input_dim=VOCAB_SIZE, activation="relu"))  # Первый полносвязный слой
model_BoW.add(Dense(86, activation="relu"))                         # Второй полносвязный слой
model_BoW.add(Dense(33, activation="relu"))                         # Третий полносвязный слой
model_BoW.add(BatchNormalization())                                 # Слой пакетной нормализации
model_BoW.add(Dropout(0.1))                                         # Слой регуляризации Dropout
model_BoW.add(Dense(CLASS_COUNT, activation='sigmoid'))             # Выходной полносвязный слой

model_BoW.compile(optimizer='adam',                                 # Компиляция модели для обучения на данных вида Bag of Words
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

Обучите модель нейронной сети и посмотрите на результаты точности на проверочной выборке:

In [ ]:
# Обучение сети с помощью функции fit()
history = model_BoW.fit(x_train_01,                            # Обучающая выборка Bag of Words
                        y_train,                               # Метки классов обучающей выборки
                        epochs=20,                             # Количество эпох
                        batch_size=32,                         # Размер подвыборки для одного шага по данным на эпохе
                        validation_data=(x_test_01, y_test))   # Проверочная выборка и метки классов проверочной выборки

Epoch 1/20
183/183 [==============================] - 6s 28ms/step - loss: 0.1087 - accuracy: 0.9798 - val_loss: 0.3222 - val_accuracy: 0.9457
Epoch 2/20
183/183 [==============================] - 4s 24ms/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 0.3091 - val_accuracy: 0.9416
Epoch 3/20
183/183 [==============================] - 4s 23ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.3145 - val_accuracy: 0.9311
Epoch 4/20
183/183 [==============================] - 4s 23ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.3007 - val_accuracy: 0.9292
Epoch 5/20
183/183 [==============================] - 4s 23ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.2501 - val_accuracy: 0.9475
Epoch 6/20
183/183 [==============================] - 4s 23ms/step - loss: 8.5893e-04 - accuracy: 1.0000 - val_loss: 0.2930 - val_accuracy: 0.9242
Epoch 7/20
183/183 [==============================] - 4s 23ms/step - loss: 6.7804e-04 - accuracy: 1.0000 - val_loss: 0.2628 - val_accuracy

Создать 9 нейросетей с различными параметрами словаря и гиперпараметрами нейронной сети.

Параметры размера словаря, ширины окна и шага:
- Размер словаря - от 10000 до 20000 (выбрать меньшее 
значение диапазона, если будет перегрузка ОЗУ и перезапуск подключения к Colaboratory)
- Ширина окна - от 1000 до 2000
- Шаг - от 100 до 500 (меньший шаг лучше для обучения, но это может перегрузить ОЗУ).




Архитектура сети с гиперпараметрами:
- Модель прямого распространения Sequential()
- Один или несколько полносвязных (Dense) слоёв
- Слои Dropout() и BatchNormalization()
- Выходной полносвязный слой с количеством нейронов, соответствующим количеству классов (число писателей)
Точность сети проверить по значению показателя 'val_accuracy'.

Запишите результаты в таблицу:

In [31]:

# Ваше задание здесь
import pandas as pd

CLASS_COUNT = len(CLASS_LIST)
VOCAB_SIZE = [10000, 15000]                      
WIN_SIZE = [1000, 2000]                       
WIN_HOP = [200, 500]
table = pd.DataFrame()   

def split_sequence(sequence, win_size, hop):
    return [sequence[i:i + win_size] for i in range(0, len(sequence) - win_size + 1, hop)]

def vectorize_sequence(seq_list, win_size, hop):
    class_count = len(seq_list)
    x, y = [], []
    
    for cls in range(class_count):
        vectors = split_sequence(seq_list[cls], win_size, hop)
        x += vectors
        y += [utils.to_categorical(cls, class_count)] * len(vectors)

    return np.array(x), np.array(y)


for i in VOCAB_SIZE:
  for j in WIN_SIZE:
    for k in WIN_HOP:
      for l in [True, False]:
        tokenizer = Tokenizer(num_words=i, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', 
                            lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)
        
        tokenizer.fit_on_texts(text_train)

        seq_train = tokenizer.texts_to_sequences(text_train)
        seq_test = tokenizer.texts_to_sequences(text_test)

        x_train, y_train = vectorize_sequence(seq_train, j, k) 
        x_test, y_test = vectorize_sequence(seq_test, j, k)

        x_train_01 = tokenizer.sequences_to_matrix(x_train.tolist())
        x_test_01 = tokenizer.sequences_to_matrix(x_test.tolist())

        model_BoW = Sequential()
        model_BoW.add(Dense(256, input_dim=i, activation="relu"))
        if l:
          model_BoW.add(Dense(128, activation="relu"))
        model_BoW.add(Dense(32, activation="relu"))
        model_BoW.add(BatchNormalization())
        model_BoW.add(Dropout(0.1))
        model_BoW.add(Dense(CLASS_COUNT, activation='sigmoid'))

        model_BoW.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        history = model_BoW.fit(x_train_01, y_train, epochs=10, batch_size=64, 
                                validation_data=(x_test_01, y_test), verbose=0)


        valAccMaxInd = np.argmax(history.history['val_accuracy'])
        # print(history.history)
        lossMax = history.history['loss'][valAccMaxInd]
        accMax = history.history['accuracy'][valAccMaxInd]
        valLossMax = history.history['val_loss'][valAccMaxInd]
        valAccMax = history.history['val_accuracy'][valAccMaxInd]

        print(np.argmax(history.history['val_accuracy']))


        historySrs = pd.Series([lossMax,accMax,valLossMax,valAccMax],
                              index=['loss','accuracy','val_loss','val_accuracy'])

        print(f'{historySrs}')
        table[f'VOCAB_SIZE:{i} WIN_SIZE:{j} WIN_HOP:{k} Sec_Dence:{l}'] = historySrs

table
print(f'Вывод: Лучший результат {round(table[table.idxmax(axis="columns")[3]]["val_accuracy"], 2)}% при параметрах {table.idxmax(axis="columns")[3]}')

1
loss            0.008170
accuracy        1.000000
val_loss        0.356853
val_accuracy    0.892344
dtype: float64
6
loss            0.000779
accuracy        1.000000
val_loss        0.311713
val_accuracy    0.904007
dtype: float64
9
loss            0.001868
accuracy        1.000000
val_loss        0.346826
val_accuracy    0.880597
dtype: float64
4
loss            0.007030
accuracy        1.000000
val_loss        0.386472
val_accuracy    0.895522
dtype: float64
8
loss            0.000540
accuracy        1.000000
val_loss        0.274876
val_accuracy    0.910380
dtype: float64
2
loss            0.003198
accuracy        1.000000
val_loss        0.309340
val_accuracy    0.924261
dtype: float64
3
loss            0.006167
accuracy        1.000000
val_loss        0.333777
val_accuracy    0.925452
dtype: float64
5
loss            0.003615
accuracy        1.000000
val_loss        0.331446
val_accuracy    0.923946
dtype: float64
2
loss            0.003075
accuracy        1.000000
val_loss    

In [33]:
table

,VOCAB_SIZE:10000 WIN_SIZE:1000 WIN_HOP:200 Sec_Dence:True,VOCAB_SIZE:10000 WIN_SIZE:1000 WIN_HOP:200 Sec_Dence:False,VOCAB_SIZE:10000 WIN_SIZE:1000 WIN_HOP:500 Sec_Dence:True,VOCAB_SIZE:10000 WIN_SIZE:1000 WIN_HOP:500 Sec_Dence:False,VOCAB_SIZE:10000 WIN_SIZE:2000 WIN_HOP:200 Sec_Dence:True,VOCAB_SIZE:10000 WIN_SIZE:2000 WIN_HOP:200 Sec_Dence:False,VOCAB_SIZE:10000 WIN_SIZE:2000 WIN_HOP:500 Sec_Dence:True,VOCAB_SIZE:10000 WIN_SIZE:2000 WIN_HOP:500 Sec_Dence:False,VOCAB_SIZE:15000 WIN_SIZE:1000 WIN_HOP:200 Sec_Dence:True,VOCAB_SIZE:15000 WIN_SIZE:1000 WIN_HOP:200 Sec_Dence:False,VOCAB_SIZE:15000 WIN_SIZE:1000 WIN_HOP:500 Sec_Dence:True,VOCAB_SIZE:15000 WIN_SIZE:1000 WIN_HOP:500 Sec_Dence:False,VOCAB_SIZE:15000 WIN_SIZE:2000 WIN_HOP:200 Sec_Dence:True,VOCAB_SIZE:15000 WIN_SIZE:2000 WIN_HOP:200 Sec_Dence:False,VOCAB_SIZE:15000 WIN_SIZE:2000 WIN_HOP:500 Sec_Dence:True,VOCAB_SIZE:15000 WIN_SIZE:2000 WIN_HOP:500 Sec_Dence:False
loss,0.008170,0.000779,0.001868,0.007030,0.000540,0.003198,0.006167,0.003615,0.003075,0.000819,0.002601,0.005212,0.000236,0.005818,0.214007,0.008708
accuracy,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.948281,1.000000
val_loss,0.356853,0.311713,0.346826,0.386472,0.274876,0.309340,0.333777,0.331446,0.377007,0.352050,0.305266,0.382290,0.235395,0.324639,0.401501,0.299937
val_accuracy,0.892344,0.904007,0.880597,0.895522,0.910380,0.924261,0.925452,0.923946,0.890550,0.885467,0.922388,0.902985,0.936331,0.930597,0.907380,0.957831
